In [ ]:
!pip install prometheus_api_client > /dev/null 2>&1
import os
import json
import hashlib
import pandas as pd
from datetime import datetime, timezone
import requests
from IPython import get_ipython
from IPython.core.interactiveshell import ExecutionResult
from prometheus_api_client import PrometheusConnect

class ExperimentTracker:
    def __init__(self):
        self.experiment_id = self._new_experiment_id()
        self.start_time = datetime.now(timezone.utc)  # Timezone-aware
        self.executed_cells = 0
        self.failed_cells = 0
        self.logs = []
        self.log_dir = f"/home/jovyan/experiment_logs/{self.experiment_id}/output/logs"
        self.metrics_dir = f"/home/jovyan/experiment_logs/{self.experiment_id}/output/metrics"
        os.makedirs(self.log_dir, exist_ok=True)
        os.makedirs(self.metrics_dir, exist_ok=True)
        print(f"[Experiment Tracking] Started experiment {self.experiment_id}")

    def export_scaph_metrics(self, start_time, end_time):
        prom_url = "https://mc-a4.lab.uvalight.net/prometheus-goncalo/"
        prom = PrometheusConnect(url=prom_url, disable_ssl=True)
        try:
            all_metrics = prom.get_label_values(label_name="__name__")
            scaph_metrics = sorted([m for m in all_metrics if m.startswith("scaph_")])
            print(f"[Experiment Tracking] Detected Scaphandre metrics: {scaph_metrics}")
        except Exception as e:
            print(f"[Experiment Tracking] Failed to fetch Prometheus metrics: {e}")
            return

        all_data = {}
        for metric in scaph_metrics:
            print(f"[Experiment Tracking] Exporting {metric}")
            try:
                data = prom.get_metric_range_data(metric_name=metric, start_time=start_time, end_time=end_time)
                if not data or "values" not in data[0]:
                    print(f"[Experiment Tracking] No data for {metric}")
                    continue
                for ts, val in data[0]["values"]:
                    dt = datetime.fromtimestamp(float(ts), tz=timezone.utc)  # Timezone-aware
                    if dt not in all_data:
                        all_data[dt] = {}
                    all_data[dt][metric] = float(val)
            except Exception as e:
                print(f"[Experiment Tracking] Error fetching data for {metric}: {e}")

        if not all_data:
            print(f"[Experiment Tracking] No Scaphandre metrics collected.")
            return

        rows = [{"timestamp": dt, **metrics} for dt, metrics in sorted(all_data.items())]
        df = pd.DataFrame(rows).sort_values("timestamp")
        csv_path = os.path.join(self.metrics_dir, "scaph_metrics_combined.csv")
        df.to_csv(csv_path, index=False)
        print(f"[Experiment Tracking] Wrote {len(df)} rows to {csv_path}")

    def _new_experiment_id(self):
        ts = datetime.utcnow().strftime("%Y%m%d%H%M%S")
        return f"exp-{hashlib.sha256(ts.encode()).hexdigest()[:8]}-{ts}"

    def pre_run_cell(self, info):
        self.cell_start_time = datetime.now(timezone.utc)  # Timezone-aware
        self.current_cell_code = info.raw_cell.strip()
        self.logs.append(f"[{self.cell_start_time}] Starting cell execution.")

    def post_run_cell(self, result: ExecutionResult):
        cell_end_time = datetime.now(timezone.utc)  # Timezone-aware
        if not hasattr(self, "cell_start_time"):
            self.cell_start_time = self.start_time
            self.current_cell_code = "<unknown cell>"
            self.logs.append(f"[{cell_end_time}] WARNING: pre_run_cell hook may not have run.")
        duration = (cell_end_time - self.cell_start_time).total_seconds()
        self.executed_cells += 1
        cell_summary = (self.current_cell_code[:60] + '...') if len(self.current_cell_code) > 60 else self.current_cell_code
        if result.error_in_exec:
            self.failed_cells += 1
            error_msg = f"[{cell_end_time}] ERROR in cell ({cell_summary}): {repr(result.error_in_exec)}"
            self.logs.append(error_msg)
            print(error_msg)
        else:
            success_msg = f"[{cell_end_time}] SUCCESS in cell ({cell_summary}) [{duration:.2f}s]"
            self.logs.append(success_msg)
            print(success_msg)

    def end_experiment(self):
        end_time = datetime.now(timezone.utc)  # Timezone-aware
        duration = (end_time - self.start_time).total_seconds()
        summary = {
            "experiment_id": self.experiment_id,
            "start_time": str(self.start_time),
            "end_time": str(end_time),
            "duration_sec": duration,
            "cells_executed": self.executed_cells,
            "cells_failed": self.failed_cells
        }
        summary_path = f"/home/jovyan/experiment_logs/{self.experiment_id}/metadata.json"
        with open(summary_path, "w") as f:
            json.dump(summary, f, indent=2)
        log_path = os.path.join(self.log_dir, "log.txt")
        with open(log_path, "w") as f:
            f.write("\n".join(self.logs))
        print(f"[Experiment Tracking] Experiment {self.experiment_id} completed and logged.")
        self.export_scaph_metrics(self.start_time, end_time)

# Register hooks.
tracker = ExperimentTracker()
ip = get_ipython()
ip.events.register('pre_run_cell', tracker.pre_run_cell)
ip.events.register('post_run_cell', tracker.post_run_cell)
print("[Experiment Tracking] IPython hooks registered.")


In [ ]:
import multiprocessing
import threading
import time
import ipywidgets as widgets
from IPython.display import display, clear_output

cancel_event = threading.Event()
stress_running = threading.Event()

def cpu_stress(cores, duration, output_box, progress_cpu):
    def cpu_load():
        while not stop_event.is_set() and not cancel_event.is_set():
            pass
    stop_event = threading.Event()
    processes = []
    try:
        output_box.append_stdout(f"Starting {cores} CPU stress processes...\n")
        for _ in range(cores):
            p = multiprocessing.Process(target=cpu_load)
            p.start()
            processes.append(p)
        for i in range(duration):
            if cancel_event.is_set():
                output_box.append_stdout("CPU stress canceled.\n")
                break
            time.sleep(1)
            progress_cpu.value = int((i + 1) / duration * 100)
    finally:
        stop_event.set()
        for p in processes:
            p.terminate()
        progress_cpu.value = 100
        output_box.append_stdout("CPU stress test ended.\n\n")

def ram_stress(mb, duration, output_box, progress_ram):
    big_data = []
    try:
        output_box.append_stdout(f"Allocating ~{mb} MB RAM...\n")
        for _ in range(mb):
            if cancel_event.is_set():
                output_box.append_stdout("RAM allocation canceled during allocation.\n")
                return
            big_data.append(bytearray(1024 * 1024))
        for i in range(duration):
            if cancel_event.is_set():
                output_box.append_stdout("RAM stress canceled.\n")
                break
            time.sleep(1)
            progress_ram.value = int((i + 1) / duration * 100)
    except MemoryError:
        output_box.append_stdout("MemoryError: Could not allocate requested memory.\n")
    finally:
        del big_data
        progress_ram.value = 100
        output_box.append_stdout("RAM stress test ended.\n\n")

def combined_stress(cores, mb, duration, output_box, progress_cpu, progress_ram):
    output_box.append_stdout("Running combined CPU and RAM stress...\n")
    cpu_thread = threading.Thread(target=cpu_stress, args=(cores, duration, output_box, progress_cpu))
    ram_thread = threading.Thread(target=ram_stress, args=(mb, duration, output_box, progress_ram))
    cpu_thread.start()
    ram_thread.start()
    cpu_thread.join()
    ram_thread.join()
    output_box.append_stdout("Combined stress test complete.\n\n")

# Widgets
warning = widgets.HTML("<b style='color: darkred;'>Warning:</b> Be mindful about increasing the pre-defined values—"
                      "setting too high may crash your environment or cause instability.")

cpu_count = multiprocessing.cpu_count()
cpu_input = widgets.BoundedIntText(value=min(2, cpu_count), min=1, max=cpu_count, layout=widgets.Layout(width='100px'))
ram_input = widgets.BoundedIntText(value=500, min=10, max=20000, step=10, layout=widgets.Layout(width='100px'))
dur_input = widgets.BoundedIntText(value=30, min=5, max=600, step=5, description="Duration (s):", layout=widgets.Layout(width='150px'))

cpu_button = widgets.Button(description="Run CPU Stress Test", button_style="warning", layout=widgets.Layout(width='150px'))
ram_button = widgets.Button(description="Run RAM Stress Test", button_style="warning", layout=widgets.Layout(width='150px'))
both_button = widgets.Button(description="Run BOTH", button_style="danger", layout=widgets.Layout(width='100%'))
cancel_button = widgets.Button(description="Cancel", button_style="info", layout=widgets.Layout(width='100%'))

output_box = widgets.Output()
progress_cpu = widgets.IntProgress(min=0, max=100, value=0, description='CPU Progress', bar_style='info', layout=widgets.Layout(width='100%'))
progress_ram = widgets.IntProgress(min=0, max=100, value=0, description='RAM Progress', bar_style='info', layout=widgets.Layout(width='100%'))

def set_buttons_state(running):
    cpu_button.disabled = running
    ram_button.disabled = running
    both_button.disabled = running
    cancel_button.disabled = not running

def run_in_thread(target):
    thread = threading.Thread(target=target)
    thread.start()

def run_cpu():
    cancel_event.clear()
    stress_running.set()
    set_buttons_state(True)
    with output_box:
        clear_output()
        progress_cpu.value = 0
        progress_ram.value = 0
    cpu_stress(cpu_input.value, dur_input.value, output_box, progress_cpu)
    stress_running.clear()
    set_buttons_state(False)

def run_ram():
    cancel_event.clear()
    stress_running.set()
    set_buttons_state(True)
    with output_box:
        clear_output()
        progress_cpu.value = 0
        progress_ram.value = 0
    ram_stress(ram_input.value, dur_input.value, output_box, progress_ram)
    stress_running.clear()
    set_buttons_state(False)

def run_both():
    cancel_event.clear()
    stress_running.set()
    set_buttons_state(True)
    with output_box:
        clear_output()
        progress_cpu.value = 0
        progress_ram.value = 0
    combined_stress(cpu_input.value, ram_input.value, dur_input.value, output_box, progress_cpu, progress_ram)
    stress_running.clear()
    set_buttons_state(False)

def on_cancel_clicked(b):
    cancel_event.set()
    with output_box:
        output_box.append_stdout("Cancellation requested.\n")

cpu_button.on_click(lambda b: run_in_thread(run_cpu))
ram_button.on_click(lambda b: run_in_thread(run_ram))
both_button.on_click(lambda b: run_in_thread(run_both))
cancel_button.on_click(on_cancel_clicked)

cpu_row = widgets.HBox([widgets.Label("CPU cores:", layout=widgets.Layout(width='80px')), cpu_input, cpu_button], layout=widgets.Layout(gap='10px'))
ram_row = widgets.HBox([widgets.Label("RAM (MB):", layout=widgets.Layout(width='80px')), ram_input, ram_button], layout=widgets.Layout(gap='10px'))

display(warning, dur_input, cpu_row, ram_row, both_button, cancel_button, progress_cpu, progress_ram, output_box)


HTML(value="<b style='color: darkred;'>Warning:</b> Be mindful about increasing the pre-defined values—setting…

Output()

In [ ]:
tracker.end_experiment()